In [ ]:
"""
Purpose: To run the soma finder on the 
new mesh segmentation

"""

# Modules for Datajoint

In [1]:
import numpy as np
import datajoint as dj
import trimesh
from tqdm.notebook import tqdm
from pathlib import Path

from os import sys
sys.path.append("/meshAfterParty/")

import datajoint_utils as du
from importlib import reload

In [2]:
import minfig
import time
import numpy as np
#want to add in a wait for the connection part
random_sleep_sec = np.random.randint(0, 30)
print(f"Sleeping {random_sleep_sec} sec before conneting")
time.sleep(random_sleep_sec)
print("Done sleeping")

du.config_celii()
du.set_minnie65_config_segmentation(minfig)
du.print_minnie65_config_paths(minfig)

#configuring will include the adapters
minnie,schema = du.configure_minnie_vm()

Sleeping 10 sec before conneting


INFO - 2020-11-22 08:05:44,601 - settings - Setting database.host to at-database.ad.bcm.edu
INFO - 2020-11-22 08:05:44,602 - settings - Setting database.user to celiib
INFO - 2020-11-22 08:05:44,603 - settings - Setting database.password to newceliipass
INFO - 2020-11-22 08:05:44,606 - settings - Setting stores to {'minnie65': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65', 'stage': '/mnt/dj-stor01/platinum/minnie65'}, 'meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/meshes'}, 'decimated_meshes': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes', 'stage': '/mnt/dj-stor01/platinum/minnie65/02/decimated_meshes'}, 'skeletons': {'protocol': 'file', 'location': '/mnt/dj-stor01/platinum/minnie65/02/skeletons'}}
INFO - 2020-11-22 08:05:44,607 - settings - Setting enable_python_native_blobs to True
INFO - 2020-11-22 08:05:44,621 - connection - Connect

Done sleeping
Current path for external_segmentation_path = /mnt/dj-stor01/platinum/minnie65/02
Current path for external_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/meshes
Current path for external_decimated_mesh_path = /mnt/dj-stor01/platinum/minnie65/02/decimated_meshes
Current path for external_skeleton_path = /mnt/dj-stor01/platinum/minnie65/02/skeletons
Connecting celiib@at-database.ad.bcm.edu:3306


INFO - 2020-11-22 08:05:44,890 - settings - Setting enable_python_native_blobs to True


# Modules for Soma Extraction

In [3]:
from soma_extraction_utils import *
import meshlab
meshlab.set_meshlab_port(current_port=None)

No port chosen so picked random port 3126


# The table that will do the soma extraction

In [4]:
#so that it will have the adapter defined
from datajoint_utils import *
from minfig.adapters import *

In [5]:
#BaylorSegmentCentroidExternal.drop()

In [6]:
decimation_version = 0
decimation_ratio = 0.25
verts_min = 10000

@schema
class BaylorSegmentCentroidExternal(dj.Computed):
    definition="""
    -> minnie.BaylorSegmentCentroid()
    ---
    centroid_x=NULL           : int unsigned                 # (EM voxels)
    centroid_y=NULL           : int unsigned                 # (EM voxels)
    centroid_z=NULL           : int unsigned                 # (EM voxels)
    n_vertices=NULL           : bigint                 #number of vertices
    n_faces=NULL            : bigint                  #number of faces
    mesh: <somas>  #datajoint adapter to get the somas mesh objects
    multiplicity=NULL         : tinyint unsigned             # the number of somas found for this base segment
    sdf=NULL                  : double                       # sdf width value for the soma
    max_side_ratio=NULL       : double                       # the maximum of the side length ratios used for check if soma
    bbox_volume_ratio=NULL    : double                       # ratio of bbox (axis aligned) volume to mesh volume to use for check if soma
    max_hole_length=NULL      : double                    #euclidean distance of the maximum hole size
    run_time=NULL : double                   # the amount of time to run (seconds)

    """

    def make(self,key):
        """
        Pseudocode: 
        1) Download all of the data from BaylorSegmentCentroid
        2) Save the mesh as an h5 py file
        3) Store the saved path as the decomposition part of the dictionary and erase the vertices and faces
        4) Insert
        
        
        """
        
        print(f"\n\n\n---- Working on {key['segment_id']}, soma idx {key['soma_index']} ----")
        #1) Download all of the data from BaylorSegmentCentroid
        soma_data = (minnie.BaylorSegmentCentroid & key).fetch1()

        #2) Save the mesh as an h5 py file
        if soma_data["soma_vertices"] is None:
            soma_data["soma_vertices"] = np.array([])
        if soma_data["soma_faces"] is None:
            soma_data["soma_faces"] = np.array([])

        returned_file_path = tu.write_h5_file(
                                            vertices=soma_data["soma_vertices"],
                                              faces=soma_data["soma_faces"],
                                              segment_id=soma_data["segment_id"],
                                              filename = f'{soma_data["segment_id"]}_{soma_data["soma_index"]}.h5',
                                                filepath=str(du.get_somas_path())
                                             )
        
        #3) Store the saved path as the decomposition part of the dictionary and erase the vertices and faces
        del soma_data["soma_vertices"]
        del soma_data["soma_faces"]
        soma_data["mesh"] = returned_file_path
        
        #4) Insert
        self.insert1(soma_data,skip_duplicates=True)

In [7]:
#(schema.jobs & "table_name='__baylor_segment_centroid_external'").delete()

In [8]:
import time
start_time = time.time()
#time.sleep(random.randint(0, 900))
print('Populate Started')
BaylorSegmentCentroidExternal.populate(reserve_jobs=True, suppress_errors=False)#, order='random')
print('Populate Done')

print(f"Total time for BaylorSegmentCentroid populate = {time.time() - start_time}")

Populate Started


INFO - 2020-11-22 08:05:57,519 - autopopulate - Found 35877 keys to populate
INFO - 2020-11-22 08:05:57,550 - connection - Transaction started
INFO - 2020-11-22 08:05:57,553 - connection - Transaction cancelled. Rolling back ...
INFO - 2020-11-22 08:05:57,599 - connection - Transaction started
INFO - 2020-11-22 08:05:57,601 - connection - Transaction cancelled. Rolling back ...
INFO - 2020-11-22 08:05:57,641 - connection - Transaction started
INFO - 2020-11-22 08:05:57,643 - connection - Transaction cancelled. Rolling back ...
INFO - 2020-11-22 08:05:57,689 - connection - Transaction started
INFO - 2020-11-22 08:05:57,696 - connection - Transaction cancelled. Rolling back ...
INFO - 2020-11-22 08:05:57,727 - connection - Transaction started
INFO - 2020-11-22 08:05:57,729 - connection - Transaction cancelled. Rolling back ...
INFO - 2020-11-22 08:05:57,750 - connection - Transaction started
INFO - 2020-11-22 08:05:57,752 - connection - Transaction cancelled. Rolling back ...
INFO - 2020




---- Working on 864691135909781417, soma idx 0 ----



---- Working on 864691135909781929, soma idx 1 ----


INFO - 2020-11-22 08:06:00,237 - connection - Transaction committed and closed.
INFO - 2020-11-22 08:06:00,271 - connection - Transaction started
INFO - 2020-11-22 08:06:00,273 - autopopulate - Populating: {'segment_id': 864691135909782441, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'soma_index': 0}
INFO - 2020-11-22 08:06:00,305 - connection - Transaction committed and closed.
INFO - 2020-11-22 08:06:00,327 - connection - Transaction started
INFO - 2020-11-22 08:06:00,328 - autopopulate - Populating: {'segment_id': 864691135909782953, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'soma_index': 0}
INFO - 2020-11-22 08:06:00,347 - connection - Transaction committed and closed.
INFO - 2020-11-22 08:06:00,380 - connection - Transaction started
INFO - 2020-11-22 08:06:00,381 - autopopulate - Populating: {'segment_id': 864691135909783209, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'soma_index': 0}
INFO - 2020-11-22 08:06:00,405 - connect




---- Working on 864691135909782441, soma idx 0 ----



---- Working on 864691135909782953, soma idx 0 ----



---- Working on 864691135909783209, soma idx 0 ----



---- Working on 864691135909783465, soma idx 0 ----


...

INFO - 2020-11-22 08:07:10,356 - connection - Transaction committed and closed.
INFO - 2020-11-22 08:07:10,390 - connection - Transaction started
INFO - 2020-11-22 08:07:10,392 - autopopulate - Populating: {'segment_id': 864691135937208580, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'soma_index': 1}
INFO - 2020-11-22 08:07:10,576 - connection - Transaction committed and closed.





---- Working on 864691135937208580, soma idx 1 ----


INFO - 2020-11-22 08:07:10,607 - connection - Transaction started
INFO - 2020-11-22 08:07:10,609 - autopopulate - Populating: {'segment_id': 864691135937209604, 'decimation_version': 0, 'decimation_ratio': Decimal('0.25'), 'soma_index': 1}
INFO - 2020-11-22 08:07:10,696 - connection - Transaction committed and closed.





---- Working on 864691135937209604, soma idx 1 ----


OperationalError: (1213, 'Deadlock found when trying to get lock; try restarting transaction')

In [ ]:
#schema.external['somas'].delete(delete_external_files=True)